# Internationalization Puzzles

In [1]:
from typing import Iterable, Generator
from urllib import request
from datetime import datetime, UTC, timedelta
from zoneinfo import ZoneInfo
from collections import defaultdict
from unicodedata import normalize, decomposition
import re


def i18in(day):
    try:
        with open(f'input/{day}') as f:
            return f.read().strip()
    except FileNotFoundError:
        r = request.Request(f'https://i18n-puzzles.com/puzzle/{day}/input')
        r.add_header('Cookie', open('../.i18ncookie').read().strip())
        r.add_header('User-Agent', 'github.com/edoannunziata/jardin')
        with open(f'input/{day}', 'bw') as f:
            f.write(request.urlopen(r).read())
        with open(f'input/{day}') as f:
            return f.read().strip()

## [Day 1 - Length limits on messaging platforms](https://i18n-puzzles.com/puzzle/1/)

In [2]:
def message_cost(m: str) -> int:
    is_sms = len(m.encode('utf8')) <= 160
    is_tweet = len(m) <= 140
    match is_sms, is_tweet:
        case True, True: return 13
        case True, False: return 11
        case False, True: return 7
        case _: return 0


messages = i18in(1).split('\n')

A = sum(message_cost(s) for s in messages)
assert A == 107989

## [Day 2 - Detecting gravitational waves](https://i18n-puzzles.com/puzzle/2/)

In [3]:
def str_to_datetime(s: str) -> datetime:
    return datetime.strptime(s[:-3] + s[-2:], '%Y-%m-%dT%H:%M:%S%z')


def get_repeated(dts: Iterable[datetime], times: int = 4):
    seen_times = defaultdict(int)
    for dt in dts:
        seen_times[dt.astimezone(UTC)] += 1
        if seen_times[dt.astimezone(UTC)] >= times:
            return dt.astimezone(UTC)


dts = map(str_to_datetime, i18in(2).split('\n'))

A = get_repeated(dts, 4).strftime("%Y-%m-%dT%H:%M:%S+00:00")
assert A == '2020-10-25T01:30:00+00:00'

## [Day 3 - Unicode passwords](https://i18n-puzzles.com/puzzle/3/)

In [4]:
def is_valid(s: str) -> bool:
    if not 4 <= len(s) <= 12: return False
    if not any(c.isdigit() for c in s): return False
    if not any(c.isupper() for c in s): return False
    if not any(c.islower() for c in s): return False
    if all(c.isascii() for c in s): return False
    return True


A = sum(is_valid(s) for s in i18in(3).split('\n'))
assert A == 509

## [Day 4 - A trip around the world](https://i18n-puzzles.com/puzzle/4/)

In [5]:
def travel_length(departure: str, arrival: str) -> timedelta:
    tz, dt = re.match(r'Departure:\s+([^\s]+)\s+(.+)', departure).groups()
    dtd = datetime.strptime(dt, '%b %d, %Y, %H:%M').replace(tzinfo=ZoneInfo(tz))
    
    tz, dt = re.match(r'Arrival:\s+([^\s]+)\s+(.+)', arrival).groups()
    dta = datetime.strptime(dt, '%b %d, %Y, %H:%M').replace(tzinfo=ZoneInfo(tz))
    
    return dta - dtd


travels = [tuple(u.split('\n')) for u in i18in(4).split('\n\n')]

A = int(
    sum((travel_length(*t) for t in travels), start=timedelta())
    .total_seconds()
    / 60
)
assert A == 16451

## [Day 5 - Don't step in it](https://i18n-puzzles.com/puzzle/5/)

In [6]:
A = sum(
    s[(2*n) % len(s)] == '\N{PILE OF POO}'
    for n, s in enumerate(i18in(5).split('\n'))
)
assert A == 74

# [Day 6 - Mojibake puzzle dictionary](https://i18n-puzzles.com/puzzle/6/)

In [7]:
def unfizzbuzz(wl: Generator[str, None, None]) -> Generator[str, None, None]:
    for i, w in enumerate(wl, 1):
        if i % 3 == 0: w = w.encode('latin1').decode('utf8')
        if i % 5 == 0: w = w.encode('latin1').decode('utf8')
        yield w

def solve_puzzle(puzzle: list[str], words: list[str]) -> Generator[tuple[int, str], None, None]:
    def get_clue(s: str) -> tuple[int, int, str] :
        letters = 0
        clue = None, None
        for n, c in enumerate(s):
            match c:
                case ' ': 
                    continue
                case '.': 
                    letters += 1
                case _: 
                    clue = (letters, c)
                    letters += 1 
                
        return letters, *clue
    
    clues = list(map(get_clue, puzzle))
   
    for n, w in enumerate(words, 1):
        for size, pos, l in clues:
            if len(w) == size and w[pos] == l:
                yield n, w


words, puzzle = i18in(6).split('\n\n')
words = words.split('\n')
puzzle = puzzle.split('\n')

A = sum(pos for pos, _ in solve_puzzle(puzzle, list(unfizzbuzz(words))))
assert A == 11252

## [Day 7 - The audit trail fixer](https://i18n-puzzles.com/puzzle/7/)

In [8]:
def fix_date(dt: datetime, a: int, b: int) -> datetime:
    halifax = ZoneInfo('America/Halifax')
    santiago = ZoneInfo('America/Santiago')
    if halifax.utcoffset(dt) == dt.utcoffset():
        home_tz = halifax
    else:
        home_tz = santiago
    t0 = dt.replace(tzinfo=home_tz) 
    t1 = t0 + timedelta(minutes=a) - timedelta(minutes=b)
    return t1 + t1.utcoffset() - t0.utcoffset()


date_lst = [u.split('\t') for u in i18in(7).split('\n')]
date_lst = [
    (
        datetime.strptime(u[0][:-3] + u[0][-2:], '%Y-%m-%dT%H:%M:%S.%f%z'),
        int(u[1]), 
        int(u[2])
    )
    for u in date_lst
]

A = sum(n * u.hour for n, u in enumerate((fix_date(dt, a, b) for dt, a, b in date_lst), 1))
assert A == 32152346

## [Day 8 - Unicode passwords redux](https://i18n-puzzles.com/puzzle/8/)

In [9]:
def is_valid(s: str) -> bool:
    if not 4 <= len(s) <= 12: return False
    s = normalize('NFKD', s).lower()
    if not any(c.isdigit() for c in s): return False
    if not any(c.isalpha() and c.isascii() and c in 'aeiou' for c in s): return False
    if not any(c.isalpha() and c.isascii() and c not in 'aeiou' for c in s): return False
    seen = set()
    for c in s:
        if c.isalpha() and c.isascii() and c in seen:
            return False
        seen.add(c)
        
    return True


A = sum(is_valid(s) for s in i18in(8).split('\n'))
assert A == 809